In [1]:
import pyomo.environ as pe
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Q3 data
unit_cost = {1: 6, 2: 16, 3: 10}
unit_price = {1: 9, 2: 15}
raw_quality = {1: 0.030, 2: 0.010, 3: 0.020}
demand = {1: 100, 2: 200}
max_quality = {1: 0.025, 2: 0.015}

In [3]:
solver = pe.SolverFactory('gurobi')

In [4]:
#Q3 P formulation
model3p = pe.ConcreteModel()
model3p.IDXfeeds = pe.RangeSet(3)
model3p.IDXpoolsources = pe.RangeSet(2)
model3p.IDXdirectsupply = {3}
model3p.IDXproducts = pe.RangeSet(2)
# idx of pools and idx of qualities are just 1
model3p.c = pe.Param(model3p.IDXfeeds, initialize=unit_cost)
model3p.d = pe.Param(model3p.IDXproducts, initialize=unit_price)
model3p.C = pe.Param(model3p.IDXfeeds, initialize=raw_quality)
model3p.D = pe.Param(model3p.IDXproducts, initialize=demand)
model3p.PU = pe.Param(model3p.IDXproducts, initialize=max_quality)

model3p.p = pe.Var(within=pe.NonNegativeReals)
# remember no raw mateiral 3 goes to pool
model3p.x = pe.Var(model3p.IDXpoolsources, within=pe.NonNegativeReals)
model3p.y = pe.Var(model3p.IDXproducts, within=pe.NonNegativeReals)
model3p.u = pe.Var(model3p.IDXproducts, within=pe.NonNegativeReals) #Binomail variable for p.y
# remember no raw mateiral 1 and 2 go to direct supply
model3p.z = pe.Var(model3p.IDXdirectsupply, model3p.IDXproducts, within=pe.NonNegativeReals)

def obj_rule(m):
    cost =  sum(m.c[i] * m.x[i] for i in m.IDXpoolsources)
    revenue = sum(m.d[j] * m.y[j] for j in m.IDXproducts) + sum((m.d[j] - m.c[i]) * m.z[i,j] for i in m.IDXdirectsupply for j in m.IDXproducts)
    return cost - revenue
model3p.obj = pe.Objective(rule=obj_rule, sense=pe.minimize)

# Conservation of flow into and out of the pools
def balance_pool_rule(m):
    return sum(m.x[i] for i in m.IDXpoolsources) == sum(m.y[j] for j in m.IDXproducts)
model3p.balance_pool = pe.Constraint(rule=balance_pool_rule)

# Sales of each product cannot exceed their demands
def demand_rule(m, j):
    return m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply) <= m.D[j]
model3p.demand = pe.Constraint(model3p.IDXproducts, rule=demand_rule)

# Maintains the quality of the product within upper bounds
def quality_rule(m, j):
    return m.u[j] + sum(m.C[i] * m.z[i, j] for i in m.IDXdirectsupply) <= m.PU[j] * (m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply))
model3p.quality = pe.Constraint(model3p.IDXproducts, rule=quality_rule)

# Balances the quality
def balance_quality_rule(m):
    return sum(m.C[i] * m.x[i] for i in m.IDXpoolsources) == sum(m.u[j] for j in m.IDXproducts)
model3p.balance_quality = pe.Constraint(rule=balance_quality_rule)

def mckormik_1(m,j):
    return m.u[j]>=m.y[j]+m.p*m.D[j]-m.D[j]
model3p.mcq1 = pe.Constraint(model3p.IDXproducts,rule=mckormik_1)

def mckormik_2(m,j):
    return m.u[j]<=m.y[j]
model3p.mcq2 = pe.Constraint(model3p.IDXproducts,rule=mckormik_2)

def mckormik_3(m,j):
    return m.u[j]<=m.D[j]*m.p
model3p.mcq3 = pe.Constraint(model3p.IDXproducts,rule=mckormik_3)



result_obj = solver.solve(model3p, tee=True)
model3p.pprint()
print(result_obj)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04
Read LP format model from file C:\Users\aramanuj\AppData\Local\Temp\tmp3qeqik13.pyomo.lp
Reading time = 0.00 seconds
x1: 12 rows, 9 columns, 32 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 9 columns and 32 nonzeros
Model fingerprint: 0x2a9a2e38
Coefficient statistics:
  Matrix range     [5e-03, 2e+02]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 11 rows, 8 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.4000000e+31   2.035000e+30   2.400000e+01      0s
       6   -5.994898

In [5]:
#Q3 Q formulation
model3q = pe.ConcreteModel()
model3q.IDXfeeds = pe.RangeSet(3)
model3q.IDXpoolsources = pe.RangeSet(2)
model3q.IDXdirectsupply = {3}
model3q.IDXproducts = pe.RangeSet(2)
# idx of pools and idx of qualities are just 1
model3q.c = pe.Param(model3q.IDXfeeds, initialize=unit_cost)
model3q.d = pe.Param(model3q.IDXproducts, initialize=unit_price)
model3q.C = pe.Param(model3q.IDXfeeds, initialize=raw_quality)
model3q.D = pe.Param(model3q.IDXproducts, initialize=demand)
model3q.PU = pe.Param(model3q.IDXproducts, initialize=max_quality)

model3q.p = pe.Var(within=pe.NonNegativeReals)
# remember no raw mateiral 3 goes to pool
model3q.q = pe.Var(model3q.IDXpoolsources, within=pe.NonNegativeReals)
model3q.y = pe.Var(model3q.IDXproducts, within=pe.NonNegativeReals)
model3q.u = pe.Var(model3q.IDXproducts,model3q.IDXpoolsources,within=pe.NonNegativeReals)  #Binomail variable for q.y
#model3q.u1 = pe.Var(model3q.IDXproducts, within=pe.NonNegativeReals) #Binomail variable for p.y
# remember no raw mateiral 1 and 2 go to direct supply
model3q.z = pe.Var(model3q.IDXdirectsupply, model3q.IDXproducts, within=pe.NonNegativeReals)

def obj_rule(m):
    # term1 = sum((sum(m.q[i] * m.y[j] * m.C[i, j] for i in m.IDXpoolsources)) * y[j] for j in m.IDXproducts)
    # term2 = sum(m.c[i] * m.x[i] for i in m.IDXpoolsources)
    cost =  sum(m.c[i] * (sum(m.u[j,i] for j in m.IDXproducts)) for i in m.IDXpoolsources)
    revenue = sum(m.d[j] * m.y[j] for j in m.IDXproducts) + \
                sum((m.d[j] - m.c[i]) * m.z[i,j] for i in m.IDXdirectsupply for j in m.IDXproducts)
    return cost - revenue
model3q.obj = pe.Objective(rule=obj_rule, sense=pe.minimize)

# Sales of each product cannot exceed their demands
def demand_rule(m, j):
    return m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply) <= m.D[j]
model3q.demand = pe.Constraint(model3q.IDXproducts, rule=demand_rule)

# Maintains the quality of the product within upper bounds
def quality_rule(m, j):
    return sum(m.C[i] * m.u[j,i] for i in m.IDXpoolsources)  + sum(m.C[i] * m.z[i, j] for i in m.IDXdirectsupply) <= m.PU[j] * (m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply))
model3q.quality = pe.Constraint(model3q.IDXproducts, rule=quality_rule)

# Balances the quality
#def balance_quality_rule(m):
    #return sum(m.C[i] * sum(m.u[j,i] for j in m.IDXproducts) for i in m.IDXpoolsources) == sum(m.u1[j] for j in m.IDXproducts)
#model3q.balance_quality = pe.Constraint(rule=balance_quality_rule)

# sum of fraction must be 1
def fraction_rule(m):
    return sum(m.q[i] for i in m.IDXpoolsources) == 1
model3q.fraction = pe.Constraint(rule=fraction_rule)

def mckormik_1(m,i,j):
    return m.u[j,i]>=m.y[j]+m.q[i]*m.D[j]-m.D[j]
model3q.mck1 = pe.Constraint(model3q.IDXpoolsources,model3q.IDXproducts,rule=mckormik_1)

def mckormik_2(m,i,j):
    return m.u[j,i]<=m.y[j]
model3q.mck2 = pe.Constraint(model3q.IDXpoolsources,model3q.IDXproducts,rule=mckormik_2)

def mckormik_3(m,i,j):
    return m.u[j,i]<=m.D[j]*m.q[i]
model3q.mck3 = pe.Constraint(model3q.IDXpoolsources,model3q.IDXproducts,rule=mckormik_3)




result_obj = solver.solve(model3q, tee=True)
model3q.pprint()
print(result_obj)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04
Read LP format model from file C:\Users\aramanuj\AppData\Local\Temp\tmp0hl45aed.pyomo.lp
Reading time = 0.00 seconds
x1: 17 rows, 10 columns, 42 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 10 columns and 42 nonzeros
Model fingerprint: 0x911180c8
Coefficient statistics:
  Matrix range     [5e-03, 2e+02]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 16 rows, 9 columns, 40 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.9000000e+03   2.500000e+01   0.000000e+00      0s
       4   -2.4500

In [6]:
#Q3 Q formulation
model3pq = pe.ConcreteModel()
model3pq.IDXfeeds = pe.RangeSet(3)
model3pq.IDXpoolsources = pe.RangeSet(2)
model3pq.IDXdirectsupply = {3}
model3pq.IDXproducts = pe.RangeSet(2)
# idx of pools and idx of qualities are just 1
model3pq.c = pe.Param(model3pq.IDXfeeds, initialize=unit_cost)
model3pq.d = pe.Param(model3pq.IDXproducts, initialize=unit_price)
model3pq.C = pe.Param(model3pq.IDXfeeds, initialize=raw_quality)
model3pq.D = pe.Param(model3pq.IDXproducts, initialize=demand)
model3pq.PU = pe.Param(model3pq.IDXproducts, initialize=max_quality)

model3pq.p = pe.Var(within=pe.NonNegativeReals)
# remember no raw mateiral 3 goes to pool
model3pq.q = pe.Var(model3pq.IDXpoolsources, within=pe.NonNegativeReals)
model3pq.y = pe.Var(model3pq.IDXproducts, within=pe.NonNegativeReals)
model3pq.u = pe.Var(model3pq.IDXproducts,model3pq.IDXpoolsources,within=pe.NonNegativeReals)
model3pq.u1 = pe.Var(model3pq.IDXproducts, within=pe.NonNegativeReals) #Binomail variable for p.y
# remember no raw mateiral 1 and 2 go to direct supply
model3pq.z = pe.Var(model3pq.IDXdirectsupply, model3pq.IDXproducts, within=pe.NonNegativeReals)

def obj_rule(m):
    # term1 = sum((sum(m.q[i] * m.y[j] * m.C[i, j] for i in m.IDXpoolsources)) * y[j] for j in m.IDXproducts)
    # term2 = sum(m.c[i] * m.x[i] for i in m.IDXpoolsources)
    cost =  sum(m.c[i] * (sum(m.u[j,i] for j in m.IDXproducts)) for i in m.IDXpoolsources)
    revenue = sum(m.d[j] * m.y[j] for j in m.IDXproducts) + \
                sum((m.d[j] - m.c[i]) * m.z[i,j] for i in m.IDXdirectsupply for j in m.IDXproducts)
    return cost - revenue
model3pq.obj = pe.Objective(rule=obj_rule, sense=pe.minimize)

# Sales of each product cannot exceed their demands
def demand_rule(m, j):
    return m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply) <= m.D[j]
model3pq.demand = pe.Constraint(model3pq.IDXproducts, rule=demand_rule)

# Maintains the quality of the product within upper bounds
def quality_rule(m, j):
    return sum(m.C[i] * m.u[j,i] for i in m.IDXpoolsources) + sum(m.C[i] * m.z[i, j] for i in m.IDXdirectsupply) <= m.PU[j] * (m.y[j] + sum(m.z[i, j] for i in m.IDXdirectsupply))
model3pq.quality = pe.Constraint(model3pq.IDXproducts, rule=quality_rule)

# Balances the quality
#def balance_quality_rule(m):
    #return sum(m.C[i] * sum(m.u[j,i] for j in m.IDXproducts) for i in m.IDXpoolsources) == sum(m.u1[j] for j in m.IDXproducts)
#model3pq.balance_quality = pe.Constraint(rule=balance_quality_rule)

# sum of fraction must be 1
def fraction_rule(m):
    return sum(m.q[i] for i in m.IDXpoolsources) == 1
model3pq.fraction = pe.Constraint(rule=fraction_rule)

def mckormik_1(m,i,j):
    return m.u[j,i]>=m.y[j]+m.q[i]*m.D[j]-m.D[j]
model3pq.mck1 = pe.Constraint(model3pq.IDXpoolsources,model3pq.IDXproducts,rule=mckormik_1)

def mckormik_2(m,i,j):
    return m.u[j,i]<=m.y[j]
model3pq.mck2 = pe.Constraint(model3pq.IDXpoolsources,model3pq.IDXproducts,rule=mckormik_2)

def mckormik_3(m,i,j):
    return m.u[j,i]<=m.D[j]*m.q[i]
model3pq.mck3 = pe.Constraint(model3pq.IDXpoolsources,model3pq.IDXproducts,rule=mckormik_3)




def redundant_rule(m, j):
    return sum(m.u[j,i] for i in m.IDXpoolsources) == m.y[j]
model3pq.red = pe.Constraint(model3pq.IDXproducts,rule=redundant_rule)
result_obj = solver.solve(model3pq, tee=True)
model3pq.pprint()
print(result_obj)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04
Read LP format model from file C:\Users\aramanuj\AppData\Local\Temp\tmpq4wj81xf.pyomo.lp
Reading time = 0.00 seconds
x1: 19 rows, 10 columns, 48 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19 rows, 10 columns and 48 nonzeros
Model fingerprint: 0x25c3db46
Coefficient statistics:
  Matrix range     [5e-03, 2e+02]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 11 rows and 3 columns
Presolve time: 0.00s
Presolved: 8 rows, 7 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.1000000e+03   4.465821e+01   0.000000e+00      0s
       4   -5.0000